In [182]:
import pandas as pd
pd.set_option('display.max_columns', 100)
datapath = 'C:\\Users\\Nick\\LendingClub Project\\New\\Data\\'
df = pd.read_csv(datapath + 'clean_loan.csv')

In [183]:
print(df.shape)
df.head()

(1303638, 87)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,30000,30000,30000.0,36 months,22.35,1151.16,D,D5,Supervisor,5 years,MORTGAGE,100000.0,Source Verified,Dec-2018,Fully Paid,n,debt_consolidation,Debt consolidation,917xx,CA,30.46,0.0,Jan-2012,0.0,11.0,1.0,15603,37.0,19.0,w,0.0,0.0,30026.442500,30026.44,30000.0,26.44,0.0,0.0,0.0,Jan-2019,30082.32,Jan-2019,0.0,1,Joint App,0.0,0.0,472330.0,42200.0,4.0,42939.0,15181.0,46.9,0.0,0.0,83.0,73.0,23.0,2.0,1.0,23.0,8.0,0.0,3.0,4.0,3.0,5.0,10.0,6.0,8.0,4.0,11.0,0.0,0.0,0.0,2.0,89.5,33.3,1.0,0.0,527120.0,98453.0,28600.0,101984.0,N,Cash,N
1,40000,40000,40000.0,60 months,16.14,975.71,C,C4,Assistant to the Treasurer (Payroll),< 1 year,MORTGAGE,45000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,456xx,OH,50.53,0.0,Jun-2009,0.0,18.0,0.0,34971,64.5,37.0,w,0.0,0.0,40856.678960,40856.68,40000.0,856.68,0.0,0.0,0.0,Feb-2019,40910.48,Feb-2019,0.0,1,Joint App,0.0,0.0,271068.0,54200.0,5.0,15059.0,14930.0,67.3,0.0,0.0,114.0,70.0,2.0,2.0,1.0,2.0,9.0,0.0,7.0,9.0,7.0,7.0,26.0,9.0,10.0,9.0,18.0,0.0,0.0,0.0,4.0,100.0,42.9,0.0,0.0,344802.0,161720.0,45700.0,167965.0,N,Cash,N
2,20000,20000,20000.0,36 months,7.56,622.68,A,A3,Teacher,10+ years,MORTGAGE,100000.0,Not Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,982xx,WA,18.92,0.0,Feb-1999,0.0,9.0,0.0,25416,29.9,19.0,w,0.0,0.0,20215.792430,20215.79,20000.0,215.79,0.0,0.0,0.0,Feb-2019,20228.39,Feb-2019,0.0,1,Joint App,0.0,0.0,515779.0,85100.0,5.0,57309.0,59684.0,29.9,0.0,0.0,171.0,238.0,1.0,1.0,5.0,1.0,13.0,0.0,5.0,5.0,5.0,6.0,5.0,5.0,9.0,5.0,9.0,0.0,0.0,0.0,1.0,94.7,20.0,0.0,0.0,622183.0,71569.0,85100.0,74833.0,N,Cash,N
3,4500,4500,4500.0,36 months,11.31,147.99,B,B3,Accounts Examiner III,10+ years,RENT,38500.0,Not Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,786xx,TX,4.64,0.0,Dec-2003,0.0,12.0,0.0,4472,15.3,25.0,w,0.0,0.0,4549.217149,4549.22,4500.0,49.22,0.0,0.0,0.0,Feb-2019,4553.46,Feb-2019,0.0,1,Individual,0.0,306.0,29137.0,29200.0,8.0,2428.0,10735.0,28.9,0.0,0.0,180.0,69.0,2.0,2.0,0.0,2.0,6.0,1.0,3.0,5.0,5.0,6.0,10.0,11.0,15.0,5.0,12.0,0.0,0.0,0.0,4.0,91.7,0.0,0.0,0.0,53795.0,29137.0,15100.0,24595.0,N,Cash,N
4,8425,8425,8425.0,36 months,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0.0,Oct-1997,0.0,21.0,0.0,36812,65.7,37.0,w,0.0,0.0,8735.148975,8735.15,8425.0,310.15,0.0,0.0,0.0,Feb-2019,8754.30,Feb-2019,0.0,1,Joint App,0.0,0.0,690320.0,56000.0,10.0,34516.0,11632.0,74.6,0.0,0.0,254.0,89.0,12.0,12.0,4.0,18.0,4.0,0.0,6.0,8.0,8.0,10.0,16.0,13.0,17.0,8.0,21.0,0.0,0.0,0.0,2.0,100.0,50.0,0.0,0.0,768304.0,189194.0,45800.0,189054.0,N,Cash,N


# Create columns for total return and return/dollar invested (ROI)

In [4]:
# First we'll drop the rows where funded_amnt_inv == 0 (1. There will be no return if there is no investment 2. Can't divide by 0 later!)
df = df[df['funded_amnt_inv'] != 0]

In [5]:
df['Return'] = df['total_pymnt_inv'] - df['funded_amnt_inv']
df['ROI'] = (df['Return']/df['funded_amnt_inv']*100).round(4)
df[['funded_amnt_inv','total_pymnt_inv','Return','ROI']]

,funded_amnt_inv,total_pymnt_inv,Return,ROI
0,30000.0,30026.44,26.44,0.0881
1,40000.0,40856.68,856.68,2.1417
2,20000.0,20215.79,215.79,1.0790
3,4500.0,4549.22,49.22,1.0938
4,8425.0,8735.15,310.15,3.6813
...,...,...,...,...
1303633,30000.0,6047.46,-23952.54,-79.8418
1303634,18000.0,5393.40,-12606.60,-70.0367
1303635,17950.0,20803.78,2853.78,15.8985
1303636,15000.0,16112.32,1112.32,7.4155


### Let's do a quick randomforestregressor to see where we are at without any data engineering

In [5]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import category_encoders as ce

features = ['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'addr_state', 'dti']
target = 'ROI'

train, test = train_test_split(df, random_state=42)
train.shape, test.shape

((977631, 89), (325878, 89))

In [6]:
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [7]:
baseline_pred = [df[target].mean()] * len(y_test)
print(f'Baseline Prediction ROI: {baseline_pred[0]:.02f}%')
print(f'Baseline Prediction MAE: {mean_absolute_error(baseline_pred, y_test)}')

Baseline Prediction ROI: 3.18%
Baseline Prediction MAE: 20.27344850417536


In [8]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    RandomForestRegressor(n_jobs=-1, n_estimators=500, random_state=42)
)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [0]:
print(f'RF MAE: {mean_absolute_error(y_pred, y_test)}')
print('Not great, we have some work to do!')

# Check through column names to see which columns are available at time of loan acceptance (NO LEAKAGE! Using the LCDataDictionary as reference)

In [7]:
df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct

### These are features that were available at time of loan and that are not duplicate or altered versions of the other features.
We will look at them individually to see what kind of engineering/encoding/imputing we may need to do to use them

In [136]:
feats = ['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'num_accts_ever_120_pd', 'num_sats']
target = 'ROI'

In [123]:
### Here we will also use a subset of the data just to define a wrangle function
x = df[feats].sample(1000).copy()

In [124]:
print(x.shape)
x.head()

(1000, 25)


,loan_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,num_accts_ever_120_pd,num_sats
1210335,6000,36 months,13.53,203.70,B5,NaN,OWN,29000.0,Source Verified,Feb-2014,debt_consolidation,NY,7.91,0.0,Aug-1988,1.0,5.0,1.0,3440,55.0,20.0,74.0,306.0,0.0,4.0
104406,6000,36 months,9.49,192.17,B2,9 years,MORTGAGE,35000.0,Verified,Jul-2016,credit_card,KS,10.73,0.0,Oct-2004,4.0,10.0,1.0,5867,24.9,58.0,140.0,104.0,0.0,10.0
766390,4800,36 months,21.45,181.96,D5,3 years,RENT,55000.0,Verified,Aug-2017,debt_consolidation,FL,10.52,1.0,Dec-2005,0.0,9.0,0.0,1804,72.2,18.0,105.0,139.0,6.0,9.0
466218,15000,36 months,12.29,500.30,C1,5 years,RENT,70000.0,Source Verified,Aug-2015,debt_consolidation,CA,6.94,0.0,Oct-2001,0.0,8.0,0.0,15844,56.2,12.0,117.0,166.0,0.0,8.0
1078451,21000,60 months,10.99,456.49,B3,9 years,MORTGAGE,68160.0,Source Verified,Oct-2014,debt_consolidation,OH,16.41,2.0,Aug-1989,0.0,14.0,0.0,23530,64.3,29.0,158.0,301.0,0.0,14.0


In [125]:
x.dtypes

loan_amnt                  int64
term                      object
int_rate                 float64
installment              float64
sub_grade                 object
emp_length                object
home_ownership            object
annual_inc               float64
verification_status       object
issue_d                   object
purpose                   object
addr_state                object
dti                      float64
delinq_2yrs              float64
earliest_cr_line          object
inq_last_6mths           float64
open_acc                 float64
pub_rec                  float64
revol_bal                  int64
revol_util               float64
total_acc                float64
mo_sin_old_il_acct       float64
mo_sin_old_rev_tl_op     float64
num_accts_ever_120_pd    float64
num_sats                 float64
dtype: object

In [168]:
x['time_since_earliest_cr_line'] = (pd.to_datetime(x['issue_d']) - pd.to_datetime(x['earliest_cr_line'])).dt.days
x['time_since_earliest_cr_line']

1210335    9315
104406     4291
766390     4261
466218     5052
1078451    9192
           ... 
265906     8735
145052     8217
5665       5387
330058     3531
617943     2769
Name: time_since_earliest_cr_line, Length: 1000, dtype: int64

In [186]:
import category_encoders as ce
def wrangle(dataframe):
    feats = ['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade', 'emp_length', 'home_ownership', 
    'annual_inc', 'verification_status', 'issue_d', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 
    'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 
    'mo_sin_old_il_acct','mo_sin_old_rev_tl_op', 'num_accts_ever_120_pd', 'num_sats', 'total_pymnt_inv', 'funded_amnt_inv']

    # Features to fillna with 0
    na_fill = ['emp_length', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'num_accts_ever_120_pd', 'num_sats', 
    'revol_util', 'dti', 'inq_last_6mths']
    
    # DF to work with
    x = dataframe[feats].copy()
    
    # Remove loans that had no investments
    x = x[x['funded_amnt_inv'] != 0]
    
    # Create ROI and Return features
    x['Return'] = x['total_pymnt_inv'] - x['funded_amnt_inv']
    x['ROI'] = (x['Return']/x['funded_amnt_inv']*100)   

    # Fix date time 
    x['days_since_earliest_cr_line'] = (pd.to_datetime(x['issue_d']) - pd.to_datetime(x['earliest_cr_line'])).dt.days
    
    # Fill nans
    x[na_fill] = x[na_fill].fillna(0)
   
    # Cat encode categorical features and remove other columns
    encoder = ce.OrdinalEncoder()
    x = encoder.fit_transform(x.drop(columns=['issue_d', 'earliest_cr_line']))
    
    return x

In [187]:
x1 = wrangle(df)
x1.shape

(1303509, 28)

In [188]:
x1.head()

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,num_accts_ever_120_pd,num_sats,total_pymnt_inv,funded_amnt_inv,Return,ROI,days_since_earliest_cr_line
0,30000,1,22.35,1151.16,1,1,1,100000.0,1,1,1,30.46,0.0,0.0,11.0,1.0,15603,37.0,19.0,83.0,73.0,0.0,11.0,30026.44,30000.0,26.44,0.088133,2526
1,40000,2,16.14,975.71,2,2,1,45000.0,2,2,2,50.53,0.0,0.0,18.0,0.0,34971,64.5,37.0,114.0,70.0,0.0,18.0,40856.68,40000.0,856.68,2.141700,3470
2,20000,1,7.56,622.68,3,3,1,100000.0,3,2,3,18.92,0.0,0.0,9.0,0.0,25416,29.9,19.0,171.0,238.0,0.0,9.0,20215.79,20000.0,215.79,1.078950,7243
3,4500,1,11.31,147.99,4,3,2,38500.0,3,2,4,4.64,0.0,0.0,12.0,0.0,4472,15.3,25.0,180.0,69.0,1.0,12.0,4549.22,4500.0,49.22,1.093778,5479
4,8425,1,27.27,345.18,5,4,1,450000.0,2,2,5,12.37,0.0,0.0,21.0,0.0,36812,65.7,37.0,254.0,89.0,0.0,21.0,8735.15,8425.0,310.15,3.681306,7731


In [189]:
x1.isnull().sum()

loan_amnt                      0
term                           0
int_rate                       0
installment                    0
sub_grade                      0
emp_length                     0
home_ownership                 0
annual_inc                     0
verification_status            0
purpose                        0
addr_state                     0
dti                            0
delinq_2yrs                    0
inq_last_6mths                 0
open_acc                       0
pub_rec                        0
revol_bal                      0
revol_util                     0
total_acc                      0
mo_sin_old_il_acct             0
mo_sin_old_rev_tl_op           0
num_accts_ever_120_pd          0
num_sats                       0
total_pymnt_inv                0
funded_amnt_inv                0
Return                         0
ROI                            0
days_since_earliest_cr_line    0
dtype: int64